## WhyNot Walkthrough

A companion notebook for the [WhyNot introduction video](https://youtu.be/QKrqJ9r7Okg).

In [1]:
import whynot as wn
import numpy as np

In [2]:
wn.world3.get_experiments()

[PollutionRCT,
 PollutionConfounding,
 PollutionUnobservedConfounding,
 PollutionMediation]

In [3]:
experiment = wn.world3.PollutionConfounding

In [4]:
experiment.description

'Study effect of intervening to decrease pollution on total population. Confounding arises becauses states with high pollution are more likely to receive treatment.'

In [5]:
# Generate an observational dataset
dataset = experiment.run(num_samples=200, seed=1234, show_progress=True)

In [6]:
# Ground truth *always* available, Y(1) - Y(0)
ate = np.mean(dataset.true_effects) / 1e8
ate

1.64438920148969

In [7]:
dataset.covariates.shape

(200, 12)

In [8]:
X, W, Y = dataset.covariates, dataset.treatments, dataset.outcomes

In [9]:
# Naive causal estimator
(np.mean(Y[W==1.0]) - np.mean(Y[W == 0.])) / 1e8

-10.254278887707137

In [10]:
# Adjustment
estimate = wn.algorithms.propensity_score_matching.estimate_treatment_effect(X, W, Y)

In [11]:
estimate.ate / 1e8

0.7831875104089451

In [15]:
experiment.get_parameters()

Params:
	Name:		treatment_bias
	Description:	Treatment probability bias between low and high pollution runs.
	Default:	0.8
	Values:		[0.5   0.625 0.75  0.875 1.   ]

In [16]:
# Weaker confounding
dataset = experiment.run(num_samples=200, seed=1234, show_progress=True, treatment_bias=0.6)

In [17]:
X, W, Y = dataset.covariates, dataset.treatments, dataset.outcomes
(np.mean(Y[W == 1.]) - np.mean(Y[W == 0.])) / 1e8

-8.200014519989967

In [18]:
# Stronger confounding
dataset = experiment.run(num_samples=200, seed=1234, show_progress=True, treatment_bias=0.95)

In [19]:
X, W, Y = dataset.covariates, dataset.treatments, dataset.outcomes
(np.mean(Y[W == 1.]) - np.mean(Y[W == 0.])) / 1e8

-13.063550247637243

In [20]:
# Causal graphs
zika_experiment = wn.zika.ZikaRCT

In [21]:
dataset = zika_experiment.run(num_samples=100, causal_graph=True)

In [24]:
dataset.causal_graph.edges

OutEdgeView([('susceptible_humans_0.0', 'susceptible_humans_1.0'), ('susceptible_humans_0.0', 'asymptomatic_humans_1.0'), ('asymptomatic_humans_0.0', 'susceptible_humans_1.0'), ('asymptomatic_humans_0.0', 'asymptomatic_humans_1.0'), ('asymptomatic_humans_0.0', 'symptomatic_humans_1.0'), ('symptomatic_humans_0.0', 'susceptible_humans_1.0'), ('symptomatic_humans_0.0', 'asymptomatic_humans_1.0'), ('symptomatic_humans_0.0', 'symptomatic_humans_1.0'), ('symptomatic_humans_0.0', 'recovered_humans_1.0'), ('symptomatic_humans_0.0', 'susceptible_mosquitos_1.0'), ('symptomatic_humans_0.0', 'exposed_mosquitos_1.0'), ('symptomatic_humans_0.0', 'human_population_1.0'), ('recovered_humans_0.0', 'susceptible_humans_1.0'), ('recovered_humans_0.0', 'recovered_humans_1.0'), ('susceptible_mosquitos_0.0', 'susceptible_mosquitos_1.0'), ('susceptible_mosquitos_0.0', 'exposed_mosquitos_1.0'), ('exposed_mosquitos_0.0', 'exposed_mosquitos_1.0'), ('exposed_mosquitos_0.0', 'infectious_mosquites_1.0'), ('infectio

In [25]:
# Reinforcement learning
import whynot.gym as gym

In [26]:
for env in gym.envs.registry.all():
    print(env.id)

Credit-v0
HIV-v0
opioid-v0
world3-v0
Zika-v0


In [27]:
env = gym.make("Zika-v0")

In [28]:
observation = env.reset()

In [29]:
observation

array([7.50e+02, 2.50e+02, 1.00e+01, 2.00e+01, 1.00e+04, 5.00e+02,
       1.00e+02, 1.03e+03, 1.06e+04])

In [30]:
action = env.action_space.sample()

In [31]:
action

array([0.4577882 , 0.5376793 , 0.13588312, 0.12280593], dtype=float32)

In [32]:
for _ in range(100):
    # Replace with your policy!
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)